In [2]:
import pandas as pd
import ast
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import json
import gc
from datetime import datetime
import re

# 1. Read dataset

In [3]:
rawdata_tourist_attraction = pd.read_excel(r"C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\AI-Driven-Tourism-Recommendation-System\Dataset\TourismRawData.xlsx", sheet_name="Tourist Attractions")
rawdata_tourist_attraction['category'] = "Tourist Attractions"

In [4]:
rawdata_restaurant = pd.read_excel(r"C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\AI-Driven-Tourism-Recommendation-System\Dataset\TourismRawData.xlsx", sheet_name="Restaurant")
rawdata_restaurant['category'] = "Restaurant"

In [5]:
rawdata_hotel = pd.read_excel(r"C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\AI-Driven-Tourism-Recommendation-System\Dataset\TourismRawData.xlsx", sheet_name="Hotel")
rawdata_hotel['category'] = "Hotel"

# 2. Remove duplicates based on place id

In [6]:
# Combine tourist attraction, restaurant, and hotel data into one DataFrame
df = pd.concat([rawdata_tourist_attraction, rawdata_restaurant, rawdata_hotel], ignore_index=True)

# Remove duplicates based on 'place_id'
df = df.drop_duplicates(subset='place_id', keep='first')

In [7]:
df.shape

(7306, 15)

# 3. Split reviews in json format into 5 separate rows

In [8]:
# Convert JSON string to list of dicts
df['reviews_data'] = df['reviews_data'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])

# Expand each list of reviews into separate rows
df_exploded = df.explode('reviews_data', ignore_index=True)

# Convert the dictionary in 'reviews' column into separate columns
df_reviews = pd.concat(
    [df_exploded.drop(columns=['reviews_data']), df_exploded['reviews_data'].apply(pd.Series)],
    axis=1
)


In [9]:
# clear variable to free up memory
del rawdata_tourist_attraction 
del rawdata_restaurant
del rawdata_hotel
del df
del df_exploded
gc.collect()

0

In [10]:
df_reviews.shape

(36530, 24)

In [11]:
df_reviews.columns

Index(['place_id', 'name', 'address', 'latitude', 'longitude', 'types',
       'googleMapsUri', 'priceRange', 'rating', 'userRatingCount',
       'operating_hours', 'extract_date', 'region', 'category', 'author_name',
       'author_url', 'language', 'original_language', 'profile_photo_url',
       'rating', 'relative_time_description', 'text', 'time', 'translated'],
      dtype='object')

# 4. Create unique identifier for the dataset by combining place_id and author_id

In [12]:
df_reviews['author_url'].head()

0    https://www.google.com/maps/contrib/106077561112931137265/reviews
1    https://www.google.com/maps/contrib/104120339735079568818/reviews
2    https://www.google.com/maps/contrib/114607057292660266052/reviews
3    https://www.google.com/maps/contrib/103490304563243170642/reviews
4    https://www.google.com/maps/contrib/114620491599698100119/reviews
Name: author_url, dtype: object

In [13]:
# Extract author_id from author_url
df_reviews['author_id'] = df_reviews['author_url'].str.split('/').str[-2]
df_reviews['author_id'].head()

0    106077561112931137265
1    104120339735079568818
2    114607057292660266052
3    103490304563243170642
4    114620491599698100119
Name: author_id, dtype: object

In [14]:
# Verify author_id no null value
df_reviews['author_id'].isnull().sum()

np.int64(0)

In [15]:
# Create uniqueID by combining place_id with author_id
df_reviews["uniqueID"] = df_reviews["place_id"] + "_" + df_reviews["author_id"]
df_reviews["uniqueID"].head()

0    ChIJoxNlLcpLzDERzE5fun6q7v0_106077561112931137265
1    ChIJoxNlLcpLzDERzE5fun6q7v0_104120339735079568818
2    ChIJoxNlLcpLzDERzE5fun6q7v0_114607057292660266052
3    ChIJoxNlLcpLzDERzE5fun6q7v0_103490304563243170642
4    ChIJoxNlLcpLzDERzE5fun6q7v0_114620491599698100119
Name: uniqueID, dtype: object

# 5. Verify reviews language

In [16]:
df_reviews['original_language'].value_counts()

original_language
en    36530
Name: count, dtype: int64

In [17]:
df_reviews['language'].value_counts()

language
en    36530
Name: count, dtype: int64

In [18]:
df_reviews['translated'].value_counts()

translated
False    36530
Name: count, dtype: int64

<br> **Findings**: <br/>
Since all values in original_language, language and translated column having same value, we will drop those columns

# 6. Drop unused columns and rename columns

In [19]:
df_reviews.columns

Index(['place_id', 'name', 'address', 'latitude', 'longitude', 'types',
       'googleMapsUri', 'priceRange', 'rating', 'userRatingCount',
       'operating_hours', 'extract_date', 'region', 'category', 'author_name',
       'author_url', 'language', 'original_language', 'profile_photo_url',
       'rating', 'relative_time_description', 'text', 'time', 'translated',
       'author_id', 'uniqueID'],
      dtype='object')

In [20]:
df_reviews = df_reviews.drop(columns=[
    'author_url', 
    'profile_photo_url', 
    'language', 
    'original_language', 
    'translated'
    ]
)

In [21]:
# Rename rating columns
# Find all columns named 'rating'
rating_indices = [i for i, col in enumerate(df_reviews.columns) if col == "rating"]

# Rename by index
df_reviews.columns.values[rating_indices[0]] = "place_overall_rating"
df_reviews.columns.values[rating_indices[1]] = "user_review_rating"

# Check result
print(df_reviews.columns)

Index(['place_id', 'name', 'address', 'latitude', 'longitude', 'types',
       'googleMapsUri', 'priceRange', 'place_overall_rating',
       'userRatingCount', 'operating_hours', 'extract_date', 'region',
       'category', 'author_name', 'user_review_rating',
       'relative_time_description', 'text', 'time', 'author_id', 'uniqueID'],
      dtype='object')


# 7. Convert unix timestamp to human readable format

In [22]:
# COnvert unix ts to human readable format timestamp yyyy-mm-dd hh:mm:ss
df_reviews["timestamp"] = pd.to_datetime(df_reviews["time"], unit='s')
df_reviews.drop(columns=['relative_time_description', 'time'], inplace= True)
df_reviews["timestamp"].head()

0   2025-04-29 23:57:15
1   2025-09-21 14:54:01
2   2025-09-07 03:59:09
3   2025-08-02 09:31:05
4   2025-04-27 08:45:35
Name: timestamp, dtype: datetime64[ns]

# 8. Extract 'state' from 'address'

In [23]:
# Remove singapore and thailand data
df_reviews = df_reviews[~df_reviews["address"].str.contains("Singapore|Thailand|Indonesia")]

In [24]:
# Extract state from address using regex 
malaysia_states = [
    "Johor", "Kedah", "Kelantan", "Kuala Lumpur", "Perak", "Pahang", "Labuan",
    "Melaka", "Malacca","Negeri Sembilan", "Pulau Pinang", "Penang", "Pantai Batu Feringghi",
    "Kota Kinabalu", "Kudat", "Manukan Island", "Borneo", "Sipadan Island", "Sandakan", "Semporna", "Sabah",
    "Perlis", "Putrajaya", "Sarawak", "Selangor", "Terengganu"
]
regex_pattern = r'(' + '|'.join(malaysia_states) + r')'
df_reviews['state'] = df_reviews['address'].str.extract(regex_pattern, flags=re.IGNORECASE)


# Standardize State 
df_reviews['state'] = df_reviews['state'].str.title()
state_map = {
    "Malacca": "Melaka",
    "Pulau Pinang": "Penang",
    "Pantai Batu Feringghi": "Penang",
    "Kota Kinabalu": "Sabah",
    "Kudat": "Sabah",
    "Manukan Island": "Sabah",
    "Borneo": "Sabah",  
    "Sipadan Island": "Sabah",
    "Sandakan": "Sabah",
    "Semporna": "Sabah"
}
df_reviews['state'].replace(state_map, inplace=True)
df_reviews.drop(columns="region", inplace=True)

# Print outcome
df_reviews['state'].value_counts()

C:\Users\Ong Hui Ling\AppData\Local\Temp\ipykernel_14660\39156487.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_reviews['state'].replace(state_map, inplace=True)


state
Kuala Lumpur       4300
Selangor           4140
Perak              3455
Johor              3010
Penang             2875
Pahang             2870
Sabah              2705
Kedah              2540
Negeri Sembilan    2170
Melaka             2120
Sarawak            1895
Terengganu         1315
Kelantan           1150
Putrajaya           990
Perlis              485
Labuan              270
Name: count, dtype: int64

In [25]:
df_reviews['state'].isnull().sum()

np.int64(10)

In [26]:
null_state = df_reviews[df_reviews['state'].isnull()]
null_state['name']

8555            Taman Negeri Perlis
8556            Taman Negeri Perlis
8557            Taman Negeri Perlis
8558            Taman Negeri Perlis
8559            Taman Negeri Perlis
9760    Sipadan Kapalai Dive Resort
9761    Sipadan Kapalai Dive Resort
9762    Sipadan Kapalai Dive Resort
9763    Sipadan Kapalai Dive Resort
9764    Sipadan Kapalai Dive Resort
Name: name, dtype: object

In [27]:
# handle special case
df_reviews['state'] = np.where(
    (df_reviews['state'].isnull()) & (df_reviews["name"] == "Taman Negeri Perlis"),  
    "Perlis",                 
    df_reviews['state']       
)

df_reviews['state'] = np.where(
    (df_reviews['state'].isnull()) & (df_reviews["name"] == "Sipadan Kapalai Dive Resort"),  
    "Sabah",                 
    df_reviews['state']       
)

In [28]:
df_reviews['state'].isnull().sum()

np.int64(0)

# 9. Check for null value

In [29]:
df_reviews.isnull().sum()

place_id                    0
name                        0
address                     0
latitude                    0
longitude                   0
types                       0
googleMapsUri               0
priceRange              22315
place_overall_rating        0
userRatingCount             0
operating_hours          4095
extract_date                0
category                    0
author_name                 0
user_review_rating          0
text                        0
author_id                   0
uniqueID                    0
timestamp                   0
state                       0
dtype: int64

In [30]:
not_null_df = df_reviews[~df_reviews['priceRange'].isnull()]

In [31]:
not_null_df['category'].value_counts()

category
Restaurant             13815
Tourist Attractions      155
Hotel                     15
Name: count, dtype: int64

In [32]:
df_reviews['category'].value_counts()

category
Restaurant             20855
Tourist Attractions    10235
Hotel                   5210
Name: count, dtype: int64

<br> **Findings**:<br/>
priceRange columns are mainly for Restaurant data, will clean that when doing analysis for each category

In [33]:
df_reviews.columns

Index(['place_id', 'name', 'address', 'latitude', 'longitude', 'types',
       'googleMapsUri', 'priceRange', 'place_overall_rating',
       'userRatingCount', 'operating_hours', 'extract_date', 'category',
       'author_name', 'user_review_rating', 'text', 'author_id', 'uniqueID',
       'timestamp', 'state'],
      dtype='object')

# 10. Extract 'category' from 'types'
Establish 4 main category
1. Tourist attraction
2. Mall
3. Hotel
4. Restaurant

In [34]:
df_reviews['types'].unique()

array(["['establishment', 'park', 'point_of_interest', 'tourist_attraction']",
       "['establishment', 'point_of_interest', 'tourist_attraction']",
       "['establishment', 'mosque', 'place_of_worship', 'point_of_interest', 'tourist_attraction']",
       "['establishment', 'point_of_interest', 'shopping_mall']",
       "['amusement_park', 'establishment', 'park', 'point_of_interest', 'tourist_attraction', 'zoo']",
       "['amusement_park', 'establishment', 'point_of_interest', 'tourist_attraction']",
       "['establishment', 'park', 'point_of_interest']",
       "['amusement_park', 'establishment', 'point_of_interest']",
       "['establishment', 'park', 'point_of_interest', 'shopping_mall']",
       "['establishment', 'museum', 'point_of_interest', 'tourist_attraction']",
       "['church', 'establishment', 'place_of_worship', 'point_of_interest', 'tourist_attraction']",
       "['establishment', 'park', 'point_of_interest', 'tourist_attraction', 'zoo']",
       "['amusement_park

In [35]:
# Apply ast.literal_eval to each string to safely convert it to a real list
# We use .drop_duplicates() first so we only process the unique strings
list_series = df_reviews['types'].drop_duplicates().apply(ast.literal_eval)

# Explode the lists and get uniques
unique_words = list_series.explode().unique()
print(unique_words)

['establishment' 'park' 'point_of_interest' 'tourist_attraction' 'mosque'
 'place_of_worship' 'shopping_mall' 'amusement_park' 'zoo' 'museum'
 'church' 'bar' 'food' 'lodging' 'restaurant' 'grocery_or_supermarket'
 'store' 'supermarket' 'art_gallery' 'home_goods_store' 'bakery'
 'travel_agency' 'aquarium' 'night_club' 'spa' 'parking' 'landmark'
 'casino' 'hindu_temple' 'natural_feature' 'pet_store' 'furniture_store'
 'campground' 'general_contractor' 'movie_theater' 'department_store'
 'clothing_store' 'health' 'cafe' 'university' 'electronics_store'
 'book_store' 'hair_care' 'pharmacy' 'rv_park' 'transit_station'
 'veterinary_care' 'hardware_store' 'real_estate_agency' 'liquor_store'
 'airport' 'bowling_alley' 'meal_takeaway' 'meal_delivery' 'bicycle_store'
 'finance' 'gym' 'storage' 'school']


<b> Findings: <br/>
We will use the unique tag from types as the checkbox/ filter for tourism recommendation algorithm

In [36]:
# Explode the lists and then get the value counts
word_counts = list_series.explode().value_counts()
print(word_counts)

types
establishment             244
point_of_interest         243
food                      133
store                      94
restaurant                 83
tourist_attraction         61
lodging                    57
cafe                       48
bar                        37
park                       29
shopping_mall              27
grocery_or_supermarket     18
amusement_park             18
museum                     16
travel_agency              15
night_club                 13
spa                        12
zoo                        12
bakery                     12
health                     12
clothing_store             11
meal_takeaway              10
place_of_worship           10
supermarket                 9
home_goods_store            8
movie_theater               8
art_gallery                 7
aquarium                    7
pet_store                   6
liquor_store                5
department_store            5
electronics_store           5
general_contractor          5
camp

In [37]:
def categorize_reviews_priority(df_reviews):
    """
    Categorizes reviews with priority: tourist_attraction > mall > hotel > restaurant
    Maintains original types for recommendation engine
    """
    
    # 1. PRIORITY ORDER (Highest to Lowest)
    PRIORITY_ORDER = [
        "tourist_attraction",  # Highest priority - if venue has tourist_attraction, it's primary
        "mall",                # Second priority  
        "hotel",               # Third priority
        "restaurant"           # Lowest priority
    ]
    
    # 2. CATEGORY KEYWORDS
    categories = {
        "tourist_attraction": [
            "tourist_attraction", "amusement_park", "museum", "park", "zoo", 
            "aquarium", "art_gallery", "landmark", "natural_feature", "campground",
            "mosque", "place_of_worship", "hindu_temple", "church" , "travel_agency",
            "university"
        ],
        "mall": [
            "shopping_mall", "department_store", "store", "movie_theater", "casino", "bowling_alley"
        ],
        "hotel": [
            "lodging", "hotel", "rv_park", "spa"
        ],
        "restaurant": [
            "restaurant", "cafe", "bakery", "food", "bar", "night_club"
        ]
    }
    
    # 3. SUBCATEGORIES (Your excellent structure)
    subcategories = {
        "tourist_attraction": {
            "museum_gallery": ["museum", "art_gallery"],
            "park_nature": ["park", "natural_feature", "campground"],
            "amusement": ["amusement_park", "zoo", "aquarium"],
            "landmark": ["landmark"],
            "religious": ["mosque", "place_of_worship", "hindu_temple", "church"]
        },
        "mall": {
            "shopping_mall": ["shopping_mall"],
            "department_store": ["department_store"],
            "entertainment_mall": ["movie_theater", "bowling_alley"]  
        },
        "hotel": {
            "hotel": ["lodging", "hotel"],
            "resort": ["rv_park", "spa", "gym"] 
        },
        "restaurant": {
            "cafe_bakery": ["cafe", "bakery"],
            "bar_nightlife": ["bar", "night_club"],
            "restaurant": ["restaurant"]
        }
    }
    
    # 4. CREATE PRIORITY LOOKUP MAP
    priority_lookup = {}
    for priority_level, category_name in enumerate(PRIORITY_ORDER):
        for keyword in categories[category_name]:
            priority_lookup[keyword] = (priority_level, category_name)
    
    # 5. MAIN CATEGORY FUNCTION (Priority-Based)
    def get_priority_main_category(types_string):
        """
        Returns the highest priority category based on types
        """
        try:
            types_list = ast.literal_eval(types_string)
            if not isinstance(types_list, list):
                types_list = []
        except (ValueError, SyntaxError):
            types_list = []
        
        # Find the highest priority category (lowest priority number)
        highest_priority_found = float('inf')  # Start with worst possible
        best_category = "other"
        
        for type_name in types_list:
            if type_name in priority_lookup:
                priority_level, category = priority_lookup[type_name]
                if priority_level < highest_priority_found:
                    highest_priority_found = priority_level
                    best_category = category
                    # Early exit if we found the highest possible priority
                    if priority_level == 0:  # tourist_attraction (highest)
                        break
        
        return best_category
    
    # 6. SUBCATEGORY FUNCTION
    def get_subcategory(types_string, main_category):
        """
        Returns the most specific subcategory within the main category
        """
        try:
            types_list = ast.literal_eval(types_string)
            if not isinstance(types_list, list):
                types_list = []
        except (ValueError, SyntaxError):
            types_list = []
        
        if main_category not in subcategories:
            return "other"
        
        # Check each subcategory in order (first match wins)
        for sub_cat, keywords in subcategories[main_category].items():
            if any(keyword in types_list for keyword in keywords):
                return sub_cat
        
        # Default subcategory if no specific match
        return f"{main_category}_general"
    
    # 7. APPLY TO DATAFRAME
    print("Applying priority-based categorization...")
    
    # Main category (priority-based)
    df_reviews['main_category'] = df_reviews['types'].apply(get_priority_main_category)
    
    # Subcategory (within main category)
    df_reviews['sub_category'] = df_reviews.apply(
        lambda row: get_subcategory(row['types'], row['main_category']), 
        axis=1
    )
    
    return df_reviews

In [38]:
# Run the categorization
df_reviews = categorize_reviews_priority(df_reviews)

Applying priority-based categorization...


In [39]:
df_reviews['main_category'].value_counts()

main_category
restaurant            19285
tourist_attraction     6880
hotel                  5865
mall                   3830
other                   440
Name: count, dtype: int64

In [40]:
# Standardize the 'category' column values
df_reviews['category'].replace(
    {'Tourist Attractions': 'tourist_attraction',
     'Restaurant': 'restaurant',
     'Hotel': 'hotel'},
    inplace=True
)

# If 'main_category' is 'other', replace it with the corresponding 'category' value
df_reviews['main_category'] = np.where(
    df_reviews['main_category'] == "other",
    df_reviews['category'],
    df_reviews['main_category']
)

# Drop category column
df_reviews.drop(columns=['category'], inplace=True)

C:\Users\Ong Hui Ling\AppData\Local\Temp\ipykernel_14660\367582030.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_reviews['category'].replace(


In [41]:
df_reviews['main_category'].value_counts()

main_category
restaurant            19330
tourist_attraction     7235
hotel                  5905
mall                   3830
Name: count, dtype: int64

In [42]:
df_reviews.to_pickle(r'C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\AI-Driven-Tourism-Recommendation-System\Dataset\clean_data1.pkl')